In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import pandas as pd

In [0]:
downloaded = drive.CreateFile({'id':'12AjcosJEQcve7PfOagugTBsdDlwcHHnZ'}) ## Load Secondary_Source.csv in the edge_list_creation folder
downloaded.GetContentFile('2_col.json')  
file_dictionary = pd.read_json('2_col.json')
del file_dictionary['Unnamed: 0']

KeyError: ignored

In [0]:
file = file_dictionary.values   ## this cell is for putting all the files in a dictionary shape.
file_dictionary = {}
for i in file:
  file_dictionary[i[0]] = i[1]

In [0]:
file_dictionary["LE TULLE v. SCOFIELD , 308 U.S. 415 (1940)"] ## Now we can check the key to be the file name, the content is matched to the key

"\n\nUnited States Supreme Court\nLE TULLE v. SCOFIELD(1940)\nNo. 63\nArgued: December 4, 1939Decided: January 2, 1940\n\n\n\n Rehearing Denied Jan. 29, 1940\n\n\n See \n309 \nU.S. 694\n, 60 S.Ct. 465, 84 L.Ed. --.\n Mr. Homer L. Bruce, of Houston, Tex., for petitioners.\n Mr. J. Louis Monarch, of Washington, D.C., for respondent. \n\n[308 \nU.S. 415, 416]\n\xa0\n\n Mr. Justice ROBERTS delivered the opinion of the Court.\n We took this case because the petition for certiorari alleged that the Circuit Court of Appeals had based its decision on a point not presented or argued by the litigants, which the petitioner had never had an opportunity to meet by the production of evidence.\n The Gulf Coast Irrigation Company was the owner of irrigation properties. Petitioner was its sole stockholder. He personally owned certain lands and other irrigation properties. November 4, 1931, the Irrigation Company, the Gulf Coast Water Company, and the petitioner, entered into an agreement which recited 

get help from: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
## This is a function designed to remove all the stopwords in our text
def preprocess(text): # one little problem, gensim.utils.simple_preprocess gets rid off "I" automatically. 
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [0]:
for key in file_dictionary:
  print("working on file "+ key)
  file_dictionary[key] = preprocess(file_dictionary[key])

Streaming output truncated to the last 5000 lines.
working on file UNITED STATES v. BODCAW CO., 440 U.S. 202 (1979)
working on file GROUP LIFE & HEALTH INS. CO. v. ROYAL DRUG CO., 440 U.S. 205 (1979)
working on file ARONSON v. QUICK POINT PENCIL CO., 440 U.S. 257 (1979)
working on file CHASE MANHATTAN BANK v. FINANCE ADMIN., 440 U.S. 447 (1979)
working on file DETROIT EDISON CO. v. NLRB, 440 U.S. 301 (1979)
working on file LAKE COUNTRY ESTATES v. TAHOE PLANNING AGCY., 440 U.S. 391 (1979)
working on file ORR v. ORR, 440 U.S. 268 (1979)
working on file ANDERS v. FLOYD, 440 U.S. 445 (1979)
working on file SCOTT v. ILLINOIS, 440 U.S. 367 (1979)
working on file RAMSEY v. NEW YORK, 440 U.S. 444 (1979)
working on file NEVADA v. HALL, 440 U.S. 410 (1979)
working on file QUERN v. JORDAN, 440 U.S. 332 (1979)
working on file NATIONAL MUFFLER DEALERS ASSN. v. UNITED STATES, 440 U.S. 472 (1979)
working on file NEW JERSEY v. PORTASH, 440 U.S. 450 (1979)
working on file NEW YORK TRANSIT AUTHORITY v. 

In [0]:
test_file = file_dictionary['LE TULLE v. SCOFIELD , 308 U.S. 415 (1940)'] # an example to show what the file is stored in the dictionrary after preprocessed.

In [0]:
processed_files = []
for key in file_dictionary:
  processed_files.append(file_dictionary[key])

In [0]:
dictionary = gensim.corpora.Dictionary(processed_files)

In [0]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [0]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_files]

In [0]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [0]:
topic_number = 20 # how many topic you want, by default 10 topics

In [0]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=topic_number, id2word=dictionary, passes=2, workers=2)

In [0]:
for idx, topic in lda_model.show_topics(num_topics=topic_number, num_words=5, log=False, formatted=True): ## change num_words to see number of words you want to see in a topic
    print('Topic: {} \nWords: {}'.format(idx+1, topic))

Topic: 1 
Words: 0.012*"trial" + 0.008*"jury" + 0.008*"defendant" + 0.007*"counsel" + 0.005*"criminal"
Topic: 2 
Words: 0.009*"commission" + 0.007*"patent" + 0.004*"agency" + 0.004*"public" + 0.003*"amendment"
Topic: 3 
Words: 0.006*"damages" + 0.006*"income" + 0.004*"fees" + 0.004*"class" + 0.003*"award"
Topic: 4 
Words: 0.008*"public" + 0.004*"water" + 0.004*"city" + 0.004*"government" + 0.003*"commission"
Topic: 5 
Words: 0.009*"commission" + 0.007*"government" + 0.006*"service" + 0.006*"rate" + 0.005*"rates"
Topic: 6 
Words: 0.005*"government" + 0.005*"lands" + 0.004*"indian" + 0.004*"districts" + 0.004*"tribe"
Topic: 7 
Words: 0.005*"judicial" + 0.004*"jurisdiction" + 0.004*"constitution" + 0.004*"government" + 0.004*"constitutional"
Topic: 8 
Words: 0.012*"plan" + 0.005*"debtor" + 0.005*"bankruptcy" + 0.004*"property" + 0.004*"provision"
Topic: 9 
Words: 0.008*"jurisdiction" + 0.008*"jury" + 0.005*"amendment" + 0.004*"public" + 0.004*"grand"
Topic: 10 
Words: 0.009*"political" + 

In [0]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx+1, topic))

Topic: 1 
Words: 0.012*"trial" + 0.008*"jury" + 0.008*"defendant" + 0.007*"counsel" + 0.005*"criminal" + 0.005*"sentence" + 0.005*"judge" + 0.004*"amendment" + 0.004*"habeas" + 0.004*"death"
Topic: 2 
Words: 0.009*"commission" + 0.007*"patent" + 0.004*"agency" + 0.004*"public" + 0.003*"amendment" + 0.003*"claims" + 0.002*"government" + 0.002*"process" + 0.002*"statutory" + 0.002*"respondents"
Topic: 3 
Words: 0.006*"damages" + 0.006*"income" + 0.004*"fees" + 0.004*"class" + 0.003*"award" + 0.003*"benefits" + 0.003*"government" + 0.003*"costs" + 0.003*"statutory" + 0.003*"property"
Topic: 4 
Words: 0.008*"public" + 0.004*"water" + 0.004*"city" + 0.004*"government" + 0.003*"commission" + 0.003*"indian" + 0.003*"clause" + 0.003*"regulation" + 0.003*"commerce" + 0.003*"property"
Topic: 5 
Words: 0.009*"commission" + 0.007*"government" + 0.006*"service" + 0.006*"rate" + 0.005*"rates" + 0.004*"jurisdiction" + 0.004*"contract" + 0.003*"board" + 0.003*"commerce" + 0.003*"public"
Topic: 6 
Word

In [0]:
top = {}
print(topic_number)
for i in range(topic_number): 
  top.update({i+1:[]})

In [0]:
top

{1: [], 2: [], 3: [], 4: [], 5: [], 6: []}

In [0]:
for key in file_dictionary:
  topics = sorted(lda_model[bow_corpus[processed_files.index(file_dictionary[key])]], key = lambda tup: -1*tup[1])
  for i in topics:
    topic_number = i[0]+1
    inside = [key, i[1]]
    if i[1] > 0.5: ## change the threshold for topic weights
      top[topic_number].append(inside)

Then we make a dictionary (top) for the topics (topics) have been generated for each document (key) in the document set (fiile_dictionary)

In [0]:
Source = []
Target = []
Source_weight = []
Target_weight = []
Topic = []

In [0]:
for i in range(1, topic_number + 1):
  print(i) 
  for j in range(len(top[i])):
    print(i, j)
    for k in range(j+1, len(top[i])):
      if top[i][j][1] >= top[i][k][1]:
        Source.append(top[i][j][0])
        Target.append(top[i][k][0])
        Source_weight.append(top[i][j][1])
        Target_weight.append(top[i][k][1])
      else:
        Source.append(top[i][k][0])
        Target.append(top[i][j][0])
        Source_weight.append(top[i][k][1])
        Target_weight.append(top[i][j][1])
      Topic.append(i)

1
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
1 42
1 43
1 44
1 45
1 46
1 47
1 48
1 49
1 50
1 51
1 52
1 53
1 54
1 55
1 56
1 57
1 58
1 59
1 60
1 61
1 62
1 63
1 64
1 65
1 66
1 67
1 68
1 69
1 70
1 71
1 72
1 73
1 74
1 75
1 76
1 77
1 78
1 79
1 80
1 81
1 82
1 83
1 84
1 85
1 86
1 87
1 88
1 89
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
1 100
1 101
1 102
1 103
1 104
1 105
1 106
1 107
1 108
1 109
1 110
1 111
1 112
1 113
1 114
1 115
1 116
1 117
1 118
1 119
1 120
1 121
1 122
1 123
1 124
1 125
1 126
1 127
1 128
1 129
1 130
1 131
1 132
1 133
1 134
1 135
1 136
1 137
1 138
1 139
1 140
1 141
1 142
1 143
1 144
1 145
1 146
1 147
1 148
1 149
1 150
1 151
1 152
1 153
1 154
1 155
1 156
1 157
1 158
1 159
1 160
1 161
1 162
1 163
1 164
1 165
1 166
1 167
1 168
1 169
1 170
1 171
1 172
1 173
1 174
1 175
1 176
1 177
1 178
1 179
1 180
1 181
1 182
1 183
1 18

In [0]:
top[15]

[['MCDONALD v. U. S., 335 U.S. 451 (1948)', 0.69029295],
 ['LUSTIG v. UNITED STATES, 338 U.S. 74 (1949)', 0.6738424],
 ['WOLF v. PEOPLE OF THE STATE OF COLO., 338 U.S. 25 (1949)', 0.5588928],
 ['WALDER v. UNITED STATES, 347 U.S. 62 (1954)', 0.53963256],
 ['MILLER v. UNITED STATES, 357 U.S. 301 (1958)', 0.9745972],
 ['DRAPER v. UNITED STATES, 358 U.S. 307 (1959)', 0.7487775],
 ['ELKINS v. UNITED STATES, 364 U.S. 206 (1960)', 0.8781329],
 ['RIOS v. UNITED STATES, 364 U.S. 253 (1960)', 0.88868093],
 ['MAPP v. OHIO, 367 U.S. 643 (1961)', 0.5465608],
 ['ROBINSON v. CALIFORNIA, 370 U.S. 660 (1962)', 0.798221],
 ['WONG SUN v. UNITED STATES, 371 U.S. 471 (1963)', 0.52127206],
 ['KER v. CALIFORNIA, 374 U.S. 23 (1963)', 0.7855755],
 ['AGUILAR v. TEXAS, 378 U.S. 108 (1964)', 0.9937181],
 ['UNITED STATES v. VENTRESCA, 380 U.S. 102 (1965)', 0.71552926],
 ['ANGELET v. FAY, 381 U.S. 654 (1965)', 0.7163855],
 ['BAXSTROM v. HEROLD, 383 U.S. 107 (1966)', 0.5406982],
 ['RIGGAN v. VIRGINIA, 384 U.S. 152 (

In [0]:
result = pd.DataFrame({'Source':Source, 'Target':Target, 'Source_Weight':Source_weight, 'Target_Weight': Target_weight, 'Topic': Topic})

In [0]:
Type = ['directed'] * len(result)

In [0]:
result = pd.DataFrame({'Source':Source, 'Target':Target, 'Source_Weight':Source_weight, 'Target_Weight': Target_weight, 'Type': Type, 'Topic': Topic})

In [0]:
result 

,Source,Target,Source_Weight,Target_Weight,Type,Topic
0,"AVERY v. STATE OF ALABAMA , 308 U.S. 444 (1940)","CHAMBERS v. STATE OF FLORIDA, 309 U.S. 227 (1940)",0.849771,0.753824,directed,1
1,"WHITE v. STATE OF TEXAS, 310 U.S. 530 (1940)","AVERY v. STATE OF ALABAMA , 308 U.S. 444 (1940)",0.989128,0.849771,directed,1
2,"AVERY v. STATE OF ALABAMA , 308 U.S. 444 (1940)","MONTGOMERY WARD & CO. v. DUNCAN, 311 U.S. 243 ...",0.849771,0.583332,directed,1
3,"WALKER v. JOHNSTON , 312 U.S. 275 (1941)","AVERY v. STATE OF ALABAMA , 308 U.S. 444 (1940)",0.953999,0.849771,directed,1
4,"SMITH v. O'GRADY , 312 U.S. 329 (1941)","AVERY v. STATE OF ALABAMA , 308 U.S. 444 (1940)",0.964937,0.849771,directed,1
...,...,...,...,...,...,...
7421423,"STURGEON v. FROST, IN HIS OFFICIAL CAPACITY AS...",FLORIDA v. GEORGIA,0.725427,0.635638,directed,6
7421424,HERRERA v. WYOMING,FLORIDA v. GEORGIA,0.831419,0.635638,directed,6
7421425,"NORTH CAROLINA, ET AL., APPELLANTS v. SANDRA...","STURGEON v. FROST, IN HIS OFFICIAL CAPACITY AS...",0.999552,0.725427,directed,6
7421426,"NORTH CAROLINA, ET AL., APPELLANTS v. SANDRA...",HERRERA v. WYOMING,0.999552,0.831419,directed,6


How many rows can a Google Sheet have?
Originally Answered: How many rows can you have in a google spreadsheet before performance becomes a serious problem? Short answer: 400,000 cells - http://docs.google.com/support/b... - Note the distinction between rows and cells. This means that you could have 40,000 rows of 10 columns or 80,000 rows of 5 columns.Oct 12, 2010

In [0]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
#removed typo in file name, needs to be My\ Drive not My Drive (backslash neccessary)

result.to_csv('topic_edge_list_1940.csv', index=False)
!cp topic_edge_list_1940.csv drive/My\ Drive/19-20/Spring/Data88/Supreme_Court #change to your directory